Spike trains are generated using univariate, self‐exciting Hawkes point process with an exponential kernel, using Ogata’s thinning algorithm. 

In [ ]:
import numpy as np
import pandas as pd

from scipy.optimize import curve_fit
from statsmodels.tsa.stattools import acf

from datetime import datetime

import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from cfg_global import project_folder_path
from calculate_tau import func_single_exp_monkey, fit_single_exp
from calculate_acf import acf_sttc
from spike_train_utils import bin_spike_train_fixed_len
os.chdir(current_wd)

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
save_folder = project_folder_path + 'results\\synthetic_data\\dataset\\'
fig_folder = project_folder_path + 'results\\synthetic_data\\dataset\\'

In [ ]:
def simulate_hawkes_thinning(rate_hz, tau_ms, alpha, duration_ms, seed=None):
    """
    Simulate 1D Hawkes with exponential kernel via Ogata's thinning.
    
    rate_hz    : stationary rate in spikes/sec
    tau_ms     : target time constant (ms)
    alpha      : kernel integral (<1 for stability)
    duration_ms: total sim time (ms)
    """
    rng = np.random.default_rng(seed)
    tau_kernel_ms = tau_ms * (1 - alpha)
    tau = tau_kernel_ms / 1000.0
    mu = rate_hz * (1 - alpha)       # baseline intensity (spikes/sec)
    
    t = 0.0
    events = []
    # current kernel sum K = sum_i (α/τ e^{-(t - t_i)/τ})
    K = 0.0
    
    # convert to seconds for intensity calculus
    T_sec = duration_ms / 1000.0
    
    while True:
        lambda_upper = mu + K             # global upper‐bound on intensity
        if lambda_upper <= 0:
            break
        # draw next candidate time increment (sec)
        w = rng.exponential(1.0 / lambda_upper)
        t += w
        if t >= T_sec:
            break
        # decay K over interval w
        K *= np.exp(-w/tau)
        # actual intensity at new t
        lambda_t = mu + K
        if rng.random() < lambda_t / lambda_upper:
            # accept
            events.append(t * 1000.0)  # store in ms
            # add kernel jump α/τ at event
            K += alpha / tau
    
    return np.array(events)

In [ ]:
rate_hz     = 1.5     # target firing rate (Hz)
tau_ms      = 100.0    # desired autocorr time constant (ms)
alpha       = 0.3      # self‐excitation weight (must be <1)

duration_ms = 3000*1    # simulate for 10 min

# bin_size_ms = 50       # bin for ACF
# maxlag_ms   = 1000      # how far to compute ACF

In [ ]:
# Generate 100 spike trains
num_trials = 100
all_spike_trains = []

for trial in range(num_trials):
    spikes = simulate_hawkes_thinning(
        rate_hz=rate_hz,
        tau_ms=tau_ms,
        alpha=alpha,
        duration_ms=duration_ms,
        seed=trial  # different seed per trial
    )
    all_spike_trains.append(spikes)

In [ ]:
# obj_array = np.array(all_spike_trains, dtype=object)
# np.save(save_folder + 'spike_trains_tau100ms_fr3_5hz_1000.npy', obj_array, allow_pickle=True)